In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sqlite3
import pandas as pd

In [ ]:
file_link = '/content/drive/My Drive/AI6Lagos/airbnb.db'

# Establish a connection to the SQLite database
connection = sqlite3.connect(file_link)

In [ ]:
cursor = connection.cursor()

In [ ]:
# Query to get the table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch the table names
table_names = cursor.fetchall()

# Print the table names
for table in table_names:
    print(table[0])

Listings
Calender


In [ ]:
# tablename
table_name = 'Listings'

#dataframe
df = pd.read_sql_query(f"SELECT * FROM {table_name};", connection)

In [ ]:
df.columns

Index(['id', 'host_id', 'host_name', 'host_since', 'host_location',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_listings_count', 'host_total_listings_count',
       'neighbourhood', 'neighbourhood_cleansed', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'first_review', 'last_review', 'license',
       'instant_bookable', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
       'City'],
      dtype='object')

In [ ]:
df.head()

,id,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,first_review,last_review,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,City
0,155548,748161,Maria,2011-06-27,"Amsterdam, Netherlands",None,NaN,100,f,1.0,...,2011-09-27,2019-08-20,0363 A562 4D7E 4A91 60DF,f,1.0,1.0,0.0,0.0,0.15,Amsterdam
1,162467,776296,Nicole,2011-07-04,"Amsterdam, Netherlands",within a day,75.0,45,f,1.0,...,2013-01-22,2023-05-26,0363 C6C3 A2DC 89AC B72B,f,1.0,1.0,0.0,0.0,0.53,Amsterdam
2,213371,1100324,Hendrik,2011-09-02,"Amsterdam, Netherlands",None,NaN,71,f,3.0,...,2011-09-13,2023-01-08,0363 F88F 4B47 6E9B 225B,f,1.0,1.0,0.0,0.0,0.16,Amsterdam
3,214531,1107389,Henry,2011-09-04,"Amsterdam, Netherlands",None,NaN,None,f,1.0,...,2011-11-07,2022-06-26,0363 1939 7BED DDCD 0353,f,1.0,0.0,1.0,0.0,1.24,Amsterdam
4,2818,3159,Daniel,2008-09-24,"Amsterdam, Netherlands",within an hour,100.0,100,None,1.0,...,2009-03-30,2023-05-31,None,f,1.0,0.0,1.0,0.0,1.95,Amsterdam


Top 10 property types with the most number of listings

In [ ]:
cursor.execute("""
SELECT property_type, COUNT(*) AS num_listings
FROM listings
GROUP BY property_type
ORDER BY num_listings DESC
LIMIT 10;""")

In [ ]:
results_task1 = cursor.fetchall()

# Print or process the results
for row in results_task1:
    print(row)

('Entire rental unit', 22596)
('Private room in rental unit', 6716)
('Entire condo', 3073)
('Entire home', 2009)
('Entire serviced apartment', 947)
('Private room in home', 774)
('Entire loft', 724)
('Private room in condo', 615)
('Private room in bed and breakfast', 573)
('Room in hotel', 532)


Most expensive property type in each city

In [ ]:
cursor.execute("""
    WITH ranked_properties AS (
        SELECT
            city,
            property_type,
            ROW_NUMBER() OVER (PARTITION BY city ORDER BY price DESC) AS rank
        FROM
            listings
    )
    SELECT
        city,
        property_type
    FROM
        ranked_properties
    WHERE
        rank = 1;
""")

In [ ]:
# Fetch the results
results_task2 = cursor.fetchall()

# Print or process the results
for row in results_task2:
    print(row)

(None, '4.0')
('Amsterdam', 'Entire home')
('Berlin', 'Room in hotel')
('Geneva', 'Entire rental unit')
('Lisbon', 'Entire rental unit')


In [ ]:
cursor.execute("""
    SELECT
        id AS listing_id,
        minimum_nights * price * reviews_per_month AS income
    FROM
        Listings
""")

In [ ]:
results_income = cursor.fetchall()

In [ ]:
cursor.execute("""
    SELECT
        id AS listing_id,
        minimum_nights * number_of_reviews AS total_nights_booked
    FROM
        Listings
""")

In [ ]:
# Fetch the results
results_nights_booked = cursor.fetchall()

In [ ]:
# Create DataFrames from the results
col_income = ["listing_id", "income"]
df_income = pd.DataFrame(results_income, columns=col_income)

In [ ]:
nights_booked_df = ["listing_id", "total_nights_booked"]
nights_booked = pd.DataFrame(results_nights_booked, columns=nights_booked_df)


In [ ]:
# Merge the two DataFrames on 'listing_id'
list_df = pd.merge(df_income, nights_booked, on="listing_id")

In [ ]:
# Retrieve the additional columns (instant_bookable, bedrooms, city)
cursor.execute("""
    SELECT
        id AS listing_id,
        instant_bookable,
        bedrooms,
        City
    FROM
        Listings
""")

In [ ]:
# Fetch the results
additional_info = cursor.fetchall()

In [ ]:
# Create a DataFrame from the additional info results
add_info = ["listing_id", "instant_bookable", "bedrooms", "City"]
df_additional_info = pd.DataFrame(additional_info, columns=add_info)

In [ ]:
# Now you can merge the two DataFrames
final_list = pd.merge(df_additional_info, df_additional_info, on="listing_id")

In [ ]:
top_10_listings = final_list.sort_values(by='listing_id', ascending=False).head(10)

In [ ]:
top_10_listings.head()

,listing_id,instant_bookable_x,bedrooms_x,City_x,instant_bookable_y,bedrooms_y,City_y
21496,within an hour,None,0.0,None,None,0.0,None
21497,within an hour,None,0.0,None,None,9.0,None
21499,within an hour,None,9.0,None,None,9.0,None
21498,within an hour,None,9.0,None,None,0.0,None
17917,within a few hours,None,16.0,None,None,16.0,None


In [ ]:
# tablename
table_name = 'Calender'

#dataframe
df1 = pd.read_sql_query(f"SELECT * FROM {table_name};", connection)

In [ ]:
df1.columns

Index(['listing_id', 'date', 'available', 'price', 'adjusted_price',
       'minimum_nights', 'maximum_nights'],
      dtype='object')

In [ ]:
df1.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,2818,2023-06-05,f,69.0,69.0,3.0,1125.0
1,2818,2023-06-06,f,69.0,69.0,3.0,1125.0
2,2818,2023-06-07,f,69.0,69.0,3.0,1125.0
3,2818,2023-06-08,f,69.0,69.0,3.0,1125.0
4,2818,2023-06-09,f,69.0,69.0,3.0,1125.0


In [ ]:
#### Challange 2

In [ ]:
cursor.execute("""
    SELECT COUNT(*) FROM (
        SELECT DISTINCT host_id
        FROM Listings
        WHERE City = 'Geneva' AND host_location = 'Geneva'
    ) AS hosts_in_geneva;
""")

# Fetch the result
result = cursor.fetchone()[0]

In [ ]:
print("Number of hosts living in the same city as their listings in Geneva:", result)

Number of hosts living in the same city as their listings in Geneva: 0


In [ ]:
cursor.execute("""
    SELECT City,
           CASE
               WHEN minimum_nights < 30 THEN 'Short-term'
               WHEN minimum_nights >= 30 THEN 'Long-term'
               ELSE 'Other'
           END AS stay_type,
           COUNT(*) AS listing_count
    FROM Listings
    GROUP BY City, stay_type;
""")

# Fetch the results
results = cursor.fetchall()

In [ ]:
# Print the results
for row in results:
    city, stay_type, listing_count = row
    print(f"City: {city}, Stay Type: {stay_type}, Listing Count: {listing_count}")

City: None, Stay Type: Long-term, Listing Count: 3
City: None, Stay Type: Other, Listing Count: 3
City: Amsterdam, Stay Type: Long-term, Listing Count: 70
City: Amsterdam, Stay Type: Short-term, Listing Count: 7538
City: Berlin, Stay Type: Long-term, Listing Count: 5600
City: Berlin, Stay Type: Short-term, Listing Count: 6871
City: Geneva, Stay Type: Long-term, Listing Count: 113
City: Geneva, Stay Type: Short-term, Listing Count: 2300
City: Lisbon, Stay Type: Long-term, Listing Count: 638
City: Lisbon, Stay Type: Short-term, Listing Count: 20697


In [ ]:
cursor.execute("""
    SELECT id,
           City,
           ROUND(
               (SUM(CASE WHEN has_availability = 't' THEN 1 ELSE 0 END) / COUNT(*) * 100.0),
               2
           ) AS availability_percentage
    FROM Listings
    WHERE id IN (38425685, 27963222, 393699, 11710930)
    GROUP BY id, City;
""")

# Fetch the results
results = cursor.fetchall()

In [ ]:
# Print the results
for row in results:
    listing_id, city, availability_percentage = row
    print(f"Listing ID: {listing_id}, City: {city}, Availability %: {availability_percentage}%")

Listing ID: 11710930, City: Lisbon, Availability %: 100.0%
Listing ID: 27963222, City: Amsterdam, Availability %: 100.0%
Listing ID: 38425685, City: Amsterdam, Availability %: 100.0%
Listing ID: 393699, City: Lisbon, Availability %: 100.0%
